# Task 1
Group number 30
Hróbjartur Höskuldsson
vdh-406

# Task 2

The ER schema can be seen here: https://ibb.co/6vhnH6W 

When designing the schema I first took a look at a possible primary key. There are a couple of attributes that 
springs to mind: Id, Url, Title and Content. These are the only fields that will not often be repeated through 
out the data set. The Url can not be repeated which might be a good option, however, the Url can change, and 
there is no way to stop a website from changing its domain for instance. The title can also not be the primary key 
since there might be two articles with the same title. Content has the same problem, although extremely unlikely, 
it might happen that two authors write the excact same article.So now we are left with Id, which will never change, 
and never be the same. Now we have found a primary key and now we set up entity sets. The schema used in the data 
set contains three entity sets. Authors, Articles, and additional_information. The reason behind this is to seperate 
the article to only include information about the content it self, not the domain or authors etc. This will help us
to look at the article it self only by the content it self. The same for Authors. We can look at the authors, 
see what, domain they post on, and their type of writing, without all the information on their specific articles. 
Since the data sets contains a list of news articles it only makes sence to have the entity set Articles to contain 
the primary key, Id. Looking at the ER model, we can see that the other two entity sets are weak entity sets. 
This is because their attributes can not distictively be a key in their own relation. So they need the help of their
host entity set, Articles. In the set Authors, we have multiple soft keys, this is because the relation of the 
relationship Wrote, will now show a better picture of their relationship. The entity set additional_information 
only contains a single soft key, this is because none of its attributes could act as a better key. And the 
relationship Contains does not necciserally show a better picture then just the entity set additional_information.
It would have been possible to have more entity sets then in this schema, but making it as simple as possible might
have helped later, while doing queries. The data set was set to a single table in our data management system and 
then seperated to multiple tables according to the ER model. 

ER Model to relations:

These are the queries used to upload create the relations from the ER model.



In [ ]:
import re
import psycopg2

connect = psycopg2.connect("dbname=testBase user=postgres password=******** host=localhost")
cur = connect.cursor()

#Entity sets:
#*articles entity set to relation*
cur.execute(
'CREATE TABLE articles'
	'AS(SELECT id, title, content, url, summary, tags, keywords'
	  'FROM news' 
	   'WHERE id between 13 and 700013)'

#*weak authors entity set to relation (includes ID)*
'CREATE TABLE authors'
	'AS(SELECT id, authors, type, domain'
	  'FROM news' 
	   'WHERE id between 13 and 700013)'
    
#*weak additional_data entity set to relation (includes ID)*
'CREATE TABLE additional_data'
	'AS(SELECT id, meta_description, meta_keywords, scraped_at, inserted_at, updated_at'
	  'FROM news' 
	   'WHERE id between 13 and 700013)'
    
# Relationships:
'CREATE TABLE wrote'
	'AS(SELECT id, title, authors, type, domain'
	  'FROM news'
	   'WHERE id between 13 and 700013)'

'CREATE TABLE contains'
	'AS(SELECT id, meta_description'
	  'FROM news' 
	   'WHERE id between 13 and 700013)'
           )

# When trying out queries, the database would often loose connection. So the amount of rows was reduced from 
# 1.000.000 to 700.000. This will let use do more complex queries without the risk of loosing connection 
# to the database.

# We can show the amount of rows in every table with the command:

cur.execute(
'select count(*)'
'from articles'
)

# Wich returns 700.001, the amount of rows in the database. We can also perform some simple queries such as:

cur.execute(
'select * from wrote'
'where type = "bias"'
)

cur.execute(
'select authors.id, authors.domain, authors.type'
'from authors' 
'INNER JOIN articles on authors.id = articles.id'
'where authors.id between 500 and 1000'
)

cur.execute(
'select * from wrote'
'where type = "bias"'
)



# Task 3

In [ ]:
# Query 1:
cur.execute(
'SELECT *'
  'FROM ('
        'SELECT  wrote.id,' 
            'wrote.domain,' 
            'additional_data.scraped_at,' 
            'ROW_NUMBER() OVER(PARTITION BY wrote.domain) rn'
        'from wrote inner join additional_data on additional_data.id = wrote.id'
	  			'where wrote.id between 13 and 700013 and wrote.type = "reliable"'
				'and scraped_at > '2018-01-15''
              ') a'
'WHERE rn = 1'
)

# Which returns:
# 96	"christianpost.com"	"2018-01-25 16:17:44.789555"	1
# 165034	"consortiumnews.com"	"2018-01-25 20:13:50.426130"	1
# 418108	"nutritionfacts.org"	"2018-01-25 20:13:50.426130"	1
# -----------------------

# Query 2:
cur.execute(
'Select authors, count(authors)''
'from wrote'
'where id between 13 and 700013 and type = 'fake''
'and authors is not NULL'
'group by authors'
'ORDER BY COUNT(*) DESC'
'LIMIT 5;'
)

# This return the five most profilic authors:
# "John Rolls"	1107
# "Zacks Investment Research"	942
# "Freedom Bunker"	859
# "Alton Parrish"	774
# "Reason Magazine"	685
# -----------------------

# Query 3:
'SELECT'
    'meta_keywords, count(*)'
'FROM additional_data'
	'where id between 13 and 700013'
	'and length(meta_keywords) > 4'
	'group by meta_keywords'
	'having count(*) >= 2'


# Task 4

In [ ]:
# Query 1:
cur.execute(
    'select *
	'from('
	'select type, count(*)'
	'from wrote'
	'where id between 13 and 10013'
	'and authors is null'
	'group by type) as q1'
	','
	'('
	'select type, count(*)'
	'from wrote'
	'where id between 13 and 700013'
	'group by type)'
	'as q2'
'where q1.type = q2.type'
'order by (q1.count* 100 / q2.count)'
)

# This query finds the amount of authors that have the assignment Null and then compares it to the total amount of 
# authors. Then it orders the result on how low the percantage is. So the type with the lowest percentage of 
# assigned authors is first and the highets on the bottom. So here we can see that satire ranked lowest with only 
# 6% of their articles have an assigned author. On the other end, the conspiracy type is 

# "satire"
# "rumor"
# "hate"
# "political"
# "fake"
# "clickbait"
# "junksci"
# "unreliable"
# "unknown"
# "bias"
# "reliable"
# "Conspiracy"
# -----------------------

# Query 2:
cur.execute(
'select wrote.type, round(avg(length(content)))'
'from articles, wrote'
'where articles.id between 13 and 100013'
'and articles.id = wrote.id'
'group by type'
'order by round'
)

# This query finds the avarage character count from articles of each type. So we can see how each type tends to have 
# different article lengths. For this query I saw right away that the query took longer than usual to compute, or 
# around 4 seconds. After experimenting with bigger amounts of articles the database would loose connection, so 
# the article amount was lowered to only 100.000 articles. So the numbers would probably differ if we had the whole 
# 1.000.000 articles, but it still shows somewhat of a pattern already since the values from 50.000 and 100.000 did 
# not seem to differ too much. Both resulting in rumour and satire having the shortest avarage article length, and 
# reliable having the longest. This might hint that the length of articles hints at the type of article it represents.

# Which returns:
# "rumor"		1962
# "satire"		1966
# "clickbait"	2254
# "hate"		2277
# “Null”		2739
# "bias"		3351
# "fake"		3355
# "unreliable"	3417
# "political"	3627
# "junksci"		4147 
# "conspiracy"	4778
# "unknown"		4782
# "reliable"	4993
# -----------------------

# Query 3:
cur.execute(
'select *'
'from	'
'	(select domain, count(*)'
'	from wrote'
'	where id between 13 and 700013'
'	and authors is not null'
'	group by domain'
'	order by count desc'
'	fetch first 100 rows only) as total'
'	,'
'	('
'	select domain, count(*)'
'		from'
'		(		select authors, domain, count(*)'
'				from wrote'
'				where id between 13 and 700013'
'		 		and authors is not null'
'				group by authors, domain) as num'
'	group by domain'
'	order by count desc) as auths'
'where auths.domain = total.domain'
'order by (auths.count * 100 / total.count)'
)

# The final query is a bit longer. It takes a look at the domains with the highest amount of articles, and sees how 
# many different authors write articles for that domain. The idea is to find if there is a corrilation between how 
# many articles a domain produces and the amount of authors that write for the domain. Note that, the author can 
# appear again if there was another author working on the same article. Also, the query does not look at Null values 
# since they are completely unknown. The result does not show any particular corrolation between the amount of 
# articles and authors a domain has. Since the domain with the highest ratio per author was thevictoryreport.org 
# with 1450 articles and only 4 authors so each author has written 362.5 articles on avarage, whereas the lowest 
# authors-to-domain ration was southfront.org with 785 articles and 703 authors so each author has written 1,1 
# articles on avarage.

# Task 5

For task five, Scrapy was used to scrape the web, and the group of letters was 'HIJKLMNOPR'.
I am not familiar with web scarping and css in general so using scrapy was quite though, since there
was not alot of clear information about its syntax online. It helped to first create a scrapy shell and find the
code needed, and then rewriting it. 
Since some of the categories have subcategories and some dont, the program should be able to navigate through all 
the pages. The solution was to have 3 different parsers. The first one start at the front page and gets the urls
of all categories from H to R. The second parser gets all urls of subcategories and urls of articles and seperates them. Then it sends the article urls to the third parser and recursively sends the subcategories to it self. That way it can scan through all the subcategories. The third parser yields all the data. The fields it produces is the title, date, content and categories.

The code is below and is excecuted in to a .csv with the following code:
scrapy crawl articles -o articles.csv

In [ ]:
import scrapy
from urllib.parse import urljoin


class QuotesSpider(scrapy.Spider):
    name = "category"
    

    start_urls = [
        'https://en.wikinews.org/wiki/Category:Politics_and_conflicts',
    ]

    def parse(self, response):
        letters = ['H','I','J','K','L','M','N','O','P']
        for auth in response.css('div.mw-category-group'):
            if auth.css('h3::text').get() in letters:
                allHrefsInCategory = auth.css('a::attr(href)').getall()
                for href in allHrefsInCategory:
                    nextPageUrl = response.urljoin(href)
                    yield scrapy.Request(url = nextPageUrl, callback = self.parse_middle)
                    
    def parse_middle(self, response):
        subcat = response.css('div[id="mw-subcategories"] a::attr(href)').getall()
        if len(subcat) > 0:
            for hrefs in subcat:
                nextPageUrl = response.urljoin(hrefs)
                yield scrapy.Request(url = nextPageUrl, callback = self.parse_middle) 
            
        for pages in response.css('div[id="mw-pages"]'):
            allHrefsInCategory = pages.css('a::attr(href)').getall()
            for href in allHrefsInCategory:
                nextPageUrl = response.urljoin(href)
                yield scrapy.Request(url = nextPageUrl, callback = self.parse_articles)                
            
                
    def parse_articles(self, response):
        yield{
                'title': response.css('h1::text').get(),
                'date': response.css('span[id="publishDate"]').xpath("@title").get(),
                'content': response.css('p::text,a[class="mw-redirect"]::text').extract(),
                'Categories': response.css('div[class="mw-normal-catlinks"] a::attr(title)').extract()
                
        }

Now that we have our .csv file we need to clean it. I used the cleaning program from the original data set and tweaked it a little. For example, the categories field needed alot of cleaning since the text had a lot of extra strings that did not contribute much. The code is below.

In [ ]:
import csv
import string
import re

def cleanNumbers(text):
    text = " "+text+" "
    for number in string.digits:
        if number in text:
            count = text.count(number)
            for x in range(count):
                if number in text:
                    placement = text.find(number)
                    if placement != -1:
                        nextIsDigit = True
                        substring = " <NUM>"
                        while nextIsDigit:
                            if text[placement + 1] in string.digits:
                                text = text[:placement] + text[1 + placement:]
                            else:
                                text = text[:placement - 1] + substring + text[1 + placement:]
                                nextIsDigit = False
    return(text)

def cleanEmails(text):
    emails = re.findall(r'[\w\.-]+@[\w\.-]+', text)
    for i in emails:
        text = text.replace(i,"<EMAIL>")  
    return text
        
def cleanUrls(text):
    urls = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', text)
    for i in urls:
        text = text.replace(i,"<URL>")  
    return text

def cleanDates(text):
    dates = re.findall(r'\d{4}-\d{2}-\d{2}', text)
    for i in dates:
        text = text.replace(i,"<DATE>")  
    dates2 = re.findall(r'\d{4}.\d{2}.\d{2}', text)
    for i in dates2:
        text = text.replace(i,"<DATE>")  
    dates3 = re.findall(r'\d{4}_\d{2}_\d{2}', text)
    for i in dates3:
        text = text.replace(i,"<DATE>")  
    return text

def cleanCategories(text):
    if text is not None:
        cats = re.findall('Categories|Category|Special, ', text)
        for i in cats:
                text = text.replace(i," ")  
        colon = re.findall(':', text)
        for i in colon:
                text = text.replace(i,",")  
    return text


def cleanText(fileName):
    r = csv.reader(open(fileName)) 
    lines = list(r) 
    id = 1
    for e in lines:
        e.insert(0, str(id))
        id = id + 1
        for x in range(len(e)):
            if "," in e[x]:
                e[x] = e[x].replace(",","")
            if "\n" in e[x]:
                e[x] = e[x].replace("\n","")
        e[3] = e[3].lower()
        e[3] = cleanDates(e[3])
        e[3] = cleanEmails(e[3])
        e[3] = cleanUrls(e[3])
        e[3] = cleanNumbers(e[3])
        if len(e) == 4:
            e.append(None)
        e[4] = cleanCategories(e[4])
    lines[0][4] = 'categories'
    
    return lines


finalText = cleanText('news.csv')
with open('scrapedCleanedNews.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(finalText)
    
print("FINISHED")



In [ ]:
# Now we only have insert the file to postgres, with the following code.
cur.execute(
'COPY newsScraped(id,title,date,content,categories)'
'FROM "/private/tmp/scrapedCleanedNews.csv" DELIMITER ',' CSV HEADER;'
)

# and we can see that it supports basic queries like:

cur.execute(
'select count(*)'
'from newsscraped'
)

# which returns 5959